In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
sampleSubmissionCSV = pd.read_csv('sample_submission.csv')

In [14]:
sampleSubmissionCSV.head()

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0


In [15]:
testSitePathTimestampData = sampleSubmissionCSV['site_path_timestamp'].str.split('_', expand=True)
testSitePathTimestampData.head(3)

,0,1,2
0,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000000009
1,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000009017
2,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000015326


In [16]:
print("testSitePathTimestampData.columns = ", testSitePathTimestampData.columns)

testSitePathTimestampData.columns =  RangeIndex(start=0, stop=3, step=1)


In [18]:
# testSites listed as separate folders under train/
trainSitesList = os.listdir('train/')
print(f"Number of train sites {len(trainSitesList)}")

uniqueTestSitesList = testSitePathTimestampData.loc[:,0].unique().tolist()
print(f"Number of test sites {len(uniqueTestSitesList)}")

Number of train sites 204
Number of test sites 24


In [20]:
uniqueTestSitesList[0:5]

['5a0546857ecc773753327266',
 '5c3c44b80379370013e0fd2b',
 '5d27075f03f801723c2e360f',
 '5d27096c03f801723c31e5e0',
 '5d27097f03f801723c320d97']